In [3]:
library(dplyr)
library(tidyr)
library(data.table)
library(clusterAnalysisUtils)

In [4]:
ucsdf <- list()
ct = 0
for (uc in snakemake@input$ucs) {
    ct = ct + 1
    ucsdf[[ct]] = clusterAnalysisUtils::clusterDataParser$new(uc,format = "UC",remove_sizes = T)$df
}

In [5]:
ucdf <- bind_rows(ucsdf)

In [6]:
swsdf <- list()
ct = 0
for (sw in snakemake@input$swarm) {
    ct = ct + 1
    swsdf[[ct]] = clusterAnalysisUtils::clusterDataParser$new(sw,format = "MT",remove_sizes = T)$df
}

In [7]:
swdf <- bind_rows(swsdf) %>%
    rename(ClusterSW=Cluster, MemberSW=Member)

In [21]:
merged <- ucdf %>%
    left_join(swdf, by=c("Cluster"="MemberSW")) %>%
    as.data.table() %>%
    select(-Cluster)

In [22]:
meta <- fread(snakemake@input$data) %>%
    select(taxon,date) %>%
    as.data.frame()
names(meta)

In [44]:
merged2 <- merged %>%
    left_join(meta, by = c("ClusterSW"="taxon")) %>%
    mutate(date = as.Date(date)) %>%
    as.data.frame()
merged2

In [49]:
mind <- min(merged2$date)
dates <- merged2$date - mind
merged2$days <- as.integer(dates)

In [53]:
merged3 <- merged2 %>%
    group_by(ClusterSW) %>%
    summarise(days  = median(days)) %>%
    as.data.frame() %>%
    rowwise() %>%
    mutate(days = mind + days) %>%
    rename(name=ClusterSW,date=days)

In [55]:
fwrite(x = merged3, file = snakemake@output$metadata, sep=",")